In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import copy

In [2]:
class Board():
    

    def __init__(self,n):
        self.n = n 
        self.directions = np.array([[a,b] for a in [-1,0,1] for b in [-1,0,1] if (a!=0 or b!=0)])
        self.pieces = np.zeros((self.n,self.n))
        
        self.pieces[self.n//2 - 1,self.n//2] = 1
        self.pieces[self.n//2,self.n//2 -1] = 1
        self.pieces[self.n//2 - 1,self.n//2 -1] = -1
        self.pieces[self.n//2,self.n//2] = -1
        
    
    def count_diff(self):
        count=0
        for square in [[a,b] for a in range(self.n) for b in range(self.n)]:
            count+= self.pieces[square[0],square[1]]            
        return count
    
    def next_square(self,square,direction):
        next_square = np.array(square) + np.array(direction)

        if (next_square[0] >= self.n or next_square[1] >= self.n) or (next_square[0] < 0 or next_square[1] < 0):
            return None
        else: return list(next_square)
            
    def get_directional_flips(self,square,direction,do_flip,color):
        flips = []

        square = self.next_square(square,direction)

        while (square!=None) and (self.pieces[square[0],square[1]] == -color):
            flips.append(square)
            square = self.next_square(square,direction)
            
        if square == None:
            flips = []
               
        if (square == None or self.pieces[square[0],square[1]] != color):
                return None
        
        if (square[0] >= self.n or square[1] >= self.n) or (square[0] < 0 or square[1] < 0):
            return None
        else:
            if flips:
                if do_flip:
                    for f in flips:
                        self.pieces[f[0],f[1]] = color
                return flips
            else: return None
     
    def can_play_square(self,square,color):
        [x,y] = square
        
        if self.pieces[x,y] != 0:
            return False

        for d in self.directions:
            b = self.get_directional_flips(square,d,False,color)
            if b:
                return True

        return False
                 
    def execute_move(self,move,color):
        s = copy.deepcopy(self)
        if s.can_play_square(move,color):
            s.pieces[move[0],move[1]] = color
            for d in self.directions:
                s.get_directional_flips(move,d,True,color)
        return s

    def get_legal_moves(self,color):
        moves = []
        for x in range(self.n):
            for y in range(self.n):
                if(self.can_play_square([x,y],color)):
                    move = [x,y]
                    moves.append(move)
        return moves
        
    def get_game_ended(self):
        
        if (len(self.get_legal_moves(1)) == 0) and (len(self.get_legal_moves(-1)) == 0):
            
            if self.count_diff() != 0:
                return np.sign(self.count_diff())
            else: return 1e-6
            
        else:
            return 0
        

                
            

In [3]:
def minimax(position,depth,player,alpha,beta):
    
    game_over = len(position.get_legal_moves(1)) == 0 and len(position.get_legal_moves(-1)) == 0
    if depth == 0 or game_over:
        return float(position.count_diff())

    if player == 1:

        max_val = -float(np.inf)
        legal_moves = position.get_legal_moves(player)
        print(player)
        print(legal_moves)
        
        children = [position.execute_move(move,player) for move in legal_moves]
        
        for child in children:
            position = child
            print(child.pieces)
            val = minimax(position,depth-1,-player,alpha,beta)
            max_val = np.max([val,max_val])
            if beta <= val:
                print('Pruned')
                break
                
            if val>alpha:
                alpha = val

        return max_val
    
    else:

        min_val = float(np.inf)
        legal_moves = position.get_legal_moves(player)
        print(player)

        print(legal_moves)
        
        children = [position.execute_move(move,player) for move in legal_moves]
        for child in children:
            position = child  
            print(position.pieces)
            val = minimax(position,depth-1,-player,alpha,beta)
            min_val = np.min([val,min_val])
            if val <= alpha:
                print('Pruned')
                break

            if val < beta:
                beta = val
                
            print(val)

        return min_val
        

In [106]:
EPS = 1e-8

class MCTS():
    def __init__(self,nnet,num_sims,board_size):
        self.board_size = board_size
        self.boards = []
        self.squares = [[a,b] for a in range(board_size) for b in range(board_size)]
        self.nnet = nnet
        self.Qsa = {} #q-values
        self.Nsa = {} # visits to s,a
        self.Ns = {} # visits to state s
        self.Ps = {} # initial policy output from nnet
        
        self.Es = {} # games ended from state s
        self.Vs = {} # valid moves for state s
        
        self.num_sims = num_sims

        
        
        
    def search(self,s,player):
            self.boards.append(s)
            if s not in self.Es:
                self.Es[s] = s.get_game_ended()
            
            if self.Es[s] != 0:
                return self.Es[s]
        
            if s not in self.Ps:
                
                self.Ps[s], v = self.nnet.predict(s,player)
        
                valid_moves = s.get_legal_moves(player)
                
                if not valid_moves:
                    valid_moves = s.get_legal_moves(-player)
                    player = -player
                    self.Ps[s], v = self.nnet.predict(s,player)

                valids = [(x in valid_moves) for x in self.squares]
                
                self.Ps[s] = self.Ps[s] * valids
                self.Ps[s] = valids/np.sum(valids)
     
            
                self.Vs[s] = valids
                self.Ns[s] = 0
                return v

                        

            valids = self.Vs[s]
            cur_best = -np.inf
            best_act = None
            
            valid_moves = s.get_legal_moves(player)
            
            if not valid_moves:
                valid_moves = s.get_legal_moves(-player)

            valids = [(x in valid_moves) for x in self.squares]
        
            for a in range(len(self.squares)):
                if valids[a]:
                    if (s,a) in self.Qsa:
                        u = self.Qsa[(s,a)] + self.Ps[s][a] * np.sqrt(self.Ns[s])/(1+self.Nsa[(s,a)])
                    
                    else:
                        u = self.Ps[s][a] * np.sqrt(self.Ns[s] + EPS)
                    
                    if u > cur_best:
                        cur_best = u
                        best_act = a
               
            a = best_act
            
            if a == None:
                print(valid_moves)
            

            move = self.squares[a]
            next_s = s.execute_move(move,player)
            next_player = -player
#             print(move,next_s.pieces)     
           
            v = self.search(next_s,next_player)
        

            if (s,a) in self.Qsa:
                self.Qsa[(s,a)] = (self.Nsa[(s,a)] * self.Qsa[(s,a)] + v)/(self.Nsa[(s,a)] + 1)
                self.Nsa[(s,a)] += 1
            
            else:
                self.Qsa[(s,a)] = v
                self.Nsa[(s,a)] = 1
            
            self.Ns[s] += 1

            return v
        
        
    def get_action_probs(self,position,player):
        
        for i in range(self.num_sims):
            self.search(position,player)
            
        s = position
       
        try:
            counts = [self.Nsa[(s,a)] if (s,a) in self.Nsa else 0 for a in range(64)]
            
        except: print(s.pieces)
        
        probs = [x/np.sum(counts) for x in counts]   
          
        return probs  
        

In [224]:
class NNet(nn.Module):
    
    def __init__(self,size):
        
        super().__init__()
 
        self.size = size
        self.conv1 = nn.Conv2d(1, 512, 3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(512, 512, 3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(512, 512, 3, stride=1)
        self.conv4 = nn.Conv2d(512, 512, 3, stride=1)

        self.bn1 = nn.BatchNorm2d(512)
        self.bn2 = nn.BatchNorm2d(512)
        self.bn3 = nn.BatchNorm2d(512)
        self.bn4 = nn.BatchNorm2d(512)

        self.fc1 = nn.Linear(512*(self.size-4)*(self.size-4), 1024)
        self.fc_bn1 = nn.LayerNorm(1024)

        self.fc2 = nn.Linear(1024, 512)
        self.fc_bn2 = nn.LayerNorm(512)

        self.fc3 = nn.Linear(512, self.size**2)

        self.fc4 = nn.Linear(512, 1)

    def forward(self, s,player):
        #                                                           s: batch_size x board_x x board_y
        s = s.view(-1, 1, self.size, self.size)                # batch_size x 1 x board_x x board_y
        s = F.relu(self.bn1(self.conv1(s)))                          # batch_size x num_channels x board_x x board_y
        s = F.relu(self.bn2(self.conv2(s)))                          # batch_size x num_channels x board_x x board_y
        s = F.relu(self.bn3(self.conv3(s)))                          # batch_size x num_channels x (board_x-2) x (board_y-2)
        s = F.relu(self.bn4(self.conv4(s)))                          # batch_size x num_channels x (board_x-4) x (board_y-4)
        s = s.view(-1, 512*(self.size-4)*(self.size-4))

        s = F.dropout(F.relu(self.fc_bn1(self.fc1(s))), p=0.3)  # batch_size x 1024
        s = F.dropout(F.relu(self.fc_bn2(self.fc2(s))), p=0.3)  # batch_size x 512

        pi = self.fc3(s)                                                                         # batch_size x action_size
        v = self.fc4(s)                                                                          # batch_size x 1

        return F.softmax(pi, dim=1), player * torch.tanh(v)
    
    
    def predict(self,position,player):
        position = torch.FloatTensor(position.pieces)
        with torch.no_grad():
            pi,v = self(position,player)
        return pi.data.cpu().numpy()[0], v.data.cpu().numpy()[0]

In [225]:
nnet = NNet(8)

In [338]:

training_examples = []
board = Board(8)
player = 1


def execute_episode(player):
    mcts = MCTS(nnet,15,8)
    training_examples = []
    board = Board(8)
    player = 1
    board = Board(8)
    squares = [[a,b] for a in range(8) for b in range(8)]
    episode_step = 0
    while True:
    
        episode_step += 1
    
        if board.get_legal_moves(player):
            pi = mcts.get_action_probs(board,player)
            action = np.random.choice(len(pi),p=pi)
            move = squares[action]
            board = board.execute_move(move,player)
            training_examples.append([board,player,pi])
        elif board.get_legal_moves(-player):
            pi = mcts.get_action_probs(board,-player)
            action = np.random.choice(len(pi),p=pi)
            move = squares[action]
            board = board.execute_move(move,-player)
            training_examples.append([board,-player,pi])
        else: 
            r = board.get_game_ended()
            if r != 0:
                print(episode_step)
                return [[x[0],x[2],x[1], r] for x in training_examples]
            
       
        player = -player


    

In [383]:
import time

In [335]:
board = Board(8)

In [363]:
%%time
train_ex = execute_episode(1)

61
CPU times: total: 36.3 s
Wall time: 13 s


In [362]:
train_ex[59][0].pieces

array([[ 1.,  1.,  1.,  1.,  1.,  1.,  1., -1.],
       [ 1.,  1.,  1., -1.,  1.,  1.,  1., -1.],
       [ 1.,  1.,  1.,  1., -1., -1.,  1., -1.],
       [ 1.,  1.,  1.,  1., -1.,  1.,  1., -1.],
       [ 1.,  1.,  1.,  1.,  1., -1.,  1., -1.],
       [-1.,  1.,  1.,  1.,  1.,  1., -1., -1.],
       [-1., -1.,  1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1.,  1.,  1.]])

In [364]:
train_examples = []

for _ in tqdm(range(100),desc='Self Play'):
        train_examples.extend(execute_episode(1))
        

Self Play:   1%|▋                                                                      | 1/100 [00:12<21:16, 12.89s/it]

61


Self Play:   2%|█▍                                                                     | 2/100 [00:25<20:49, 12.75s/it]

61


Self Play:   3%|██▏                                                                    | 3/100 [00:37<20:21, 12.60s/it]

...

61


Self Play:  99%|█████████████████████████████████████████████████████████████████████▎| 99/100 [25:04<00:16, 16.61s/it]

61


Self Play: 100%|█████████████████████████████████████████████████████████████████████| 100/100 [25:20<00:00, 15.21s/it]

61


In [376]:
samples = [[b[0].pieces,b[1],b[2],b[3]] for b in train_examples]

In [377]:
len(samples)

5998

In [128]:
pnet = copy.deepcopy(nnet)

In [382]:
nnet = NNet(8)

In [384]:
def train(nnet,samples,epochs,batch_size):
    optimizer = optim.Adam(nnet.parameters())
    loss_pi_fn = nn.CrossEntropyLoss()
    loss_v_fn = nn.MSELoss()
    
    for epoch in range(epochs):
        pi_losses = 0.0
        v_losses = 0.0
        
        batch_count = int(len(samples)/batch_size)
        
        t = tqdm(range(batch_count),desc='Training Net')
        for _ in t:
            sample_ids = np.random.randint(len(samples),size=batch_size)
            boards,pis,players,vs = list(zip(*[samples[i] for i in sample_ids]))
            target_pis = torch.FloatTensor(np.array(pis))
            boards = torch.FloatTensor(np.array(boards))
            target_vs = torch.FloatTensor(np.array(vs))
            players = torch.FloatTensor(players).unsqueeze(1)
            
            
            out_pi, out_v = nnet(boards,players)

            
            loss_pi = loss_pi_fn(target_pis,out_pi)
            loss_v = loss_v_fn(target_vs,out_v)
            
            total_loss = loss_pi + loss_v
            
            optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()
            
            print(total_loss)

In [385]:
train(nnet,samples,3,16)

Training Net:   0%|▏                                                                   | 1/374 [00:00<01:29,  4.15it/s]

tensor(5.4157, grad_fn=<AddBackward0>)


Training Net:   1%|▎                                                                   | 2/374 [00:00<01:30,  4.10it/s]

tensor(6.2236, grad_fn=<AddBackward0>)


Training Net:   1%|▌                                                                   | 3/374 [00:00<01:33,  3.95it/s]

...

tensor(5.1126, grad_fn=<AddBackward0>)


Training Net: 100%|█████████████████████████████████████████████████████████████████▊| 373/374 [02:08<00:00,  2.81it/s]

tensor(5.1679, grad_fn=<AddBackward0>)


Training Net: 100%|██████████████████████████████████████████████████████████████████| 374/374 [02:08<00:00,  2.90it/s]

tensor(5.1731, grad_fn=<AddBackward0>)


In [386]:
pmcts = MCTS(pnet,15,8)

In [396]:
nmcts = MCTS(nnet,15,8)

In [397]:
def play_game():
        
        players = [1,-1]
        
        position = Board(8)
        squares = [[a,b] for a in range(8) for b in range(8)]
        
        tree = [nmcts,pmcts]
        
        cur_player = 1
        cur_tree = 0
        moves = 0
        
        while position.get_game_ended() == 0:
            
            moves += 1
  
            
            action_idx = np.argmax(tree[cur_tree].get_action_probs(position,cur_player))
            
            move = squares[action_idx]
            
            position = position.execute_move(move,cur_player) 
            
            cur_player = -cur_player
            cur_tree = (cur_tree+1) % 2
            
        return position.get_game_ended()
            
        
                                        
            
            
            
            
    

In [399]:
def play_games(num_games):
    
    num = num_games//2
    
    player_1 = 0
    player_2 = 0
    draws = 0
    
    for _ in tqdm(range(num),desc='Games'):
        
        game_result = play_game()
        
        if game_result == 1:
            player_1 += 1
            
        elif game_result == -1:
            player_2 += 1
            
        else:
            draws += 0
            
            
    for _ in tqdm(range(num),desc='Games'):
        
        game_result = play_game()
        
        if game_result == 1:
            player_2 += 1
            
        elif game_result == -1:
            player_1 += 1
            
        else:
            draws += 0
            
    
    return player_1,player_2, draws    

In [400]:
play_games(10)

Games: 100%|█████████████████████████████████████████████████████████████████████████████| 5/5 [01:08<00:00, 13.80s/it]


(6, 4, 0)